In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import gc
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    get_scheduler,
    BitsAndBytesConfig
)
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

# Explicitly import bitsandbytes correctly
try:
    import bitsandbytes as bnb
except ImportError:
    raise ImportError()

def clear_gpu_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    print(f"Memory cleared. GPU memory: {torch.cuda.memory_allocated() // 1024**2} MB")

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    if torch.cuda.is_available():
        print(f"CUDA Version: {torch.version.cuda}")
        print(f"BitsAndBytes Version: {bnb.__version__}")
        print(f"Initial GPU memory: {torch.cuda.memory_allocated() // 1024**2} MB")

    # Define model identifiers from the Hugging Face model hub
    teacher_model_id = "meta-llama/llama-3.2-3b"  # Replace with actual identifier
    student_model_id = "meta-llama/llama-3.2-1b"  # Replace with actual identifier

    # Create 8-bit quantization configuration
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch.float16
    )

    # Load the tokenizer first
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(teacher_model_id)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Load teacher model with BitsAndBytesConfig in evaluation mode
    print("Loading teacher model in 8-bit...")
    teacher_model = AutoModelForCausalLM.from_pretrained(
        teacher_model_id,
        quantization_config=quantization_config,
        device_map="auto"
    )
    teacher_model.eval()  # Set to evaluation mode
    print(f"Teacher model loaded. GPU memory: {torch.cuda.memory_allocated() // 1024**2} MB")

    # Load student model with BitsAndBytesConfig in training mode
    print("Loading student model in 8-bit...")
    student_model = AutoModelForCausalLM.from_pretrained(
        student_model_id,
        quantization_config=quantization_config,
        device_map="auto"
    )
    student_model.train()  # Set to training mode
    print(f"Student model loaded. GPU memory: {torch.cuda.memory_allocated() // 1024**2} MB")

    # Load a sample dataset (using WikiText-2 as an example)
    dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:5%]")  # Use smaller portion

    # Tokenize the dataset; adjust max_length as needed
    def tokenize_function(example):
        return tokenizer(example["text"], truncation=True, max_length=256)

    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

    # Use a data collator for causal language modeling
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Create a DataLoader with smaller batch size
    batch_size = 2  # Adjust based on available memory
    dataloader = DataLoader(
        tokenized_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=data_collator
    )

    # Define optimizer using BitsAndBytes 8-bit optimizer
    print("Setting up 8-bit optimizer...")
    optimizer = bnb.optim.AdamW8bit(
        student_model.parameters(),
        lr=5e-5,
        betas=(0.9, 0.999),
        eps=1e-8,
        weight_decay=0.01
    )

    num_epochs = 2
    total_steps = num_epochs * len(dataloader)

    # Set up scheduler
    scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=total_steps // 10,
        num_training_steps=total_steps
    )

    # Temperature for softening logits during distillation
    temperature = 2.0
    kl_loss_fct = nn.KLDivLoss(reduction="batchmean")

    # Training loop for distillation
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}")

        for step, batch in enumerate(progress_bar):
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            with torch.no_grad():
                teacher_outputs = teacher_model(input_ids=input_ids, attention_mask=attention_mask)
                teacher_logits = teacher_outputs.logits / temperature
                del teacher_outputs
            student_outputs = student_model(input_ids=input_ids, attention_mask=attention_mask)
            student_logits = student_outputs.logits / temperature
            del student_outputs

            loss = kl_loss_fct(
                F.log_softmax(student_logits, dim=-1),
                F.softmax(teacher_logits, dim=-1)
            ) * (temperature ** 2)

            del teacher_logits
            del student_logits

            loss.backward()
            optimizer.step()
            scheduler.step()

            epoch_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        avg_loss = epoch_loss / len(dataloader)
        print(f"Epoch {epoch+1} average loss: {avg_loss:.4f}")
        print(f"GPU memory after epoch: {torch.cuda.memory_allocated() // 1024**2} MB")

    print("Saving 8-bit model...")

    student_model.save_pretrained("distilled_llama_1b_8bit", safe_serialization=True)
    tokenizer.save_pretrained("distilled_llama_1b_8bit")

    with open("distilled_llama_1b_8bit/quantization_config.json", "w") as f:
        f.write(quantization_config.to_json_string())

    print("Distilled model saved as 'distilled_llama_1b_8bit'.")

    del teacher_model
    del student_model
    clear_gpu_memory()
    print("Process completed successfully.")

if __name__ == "__main__":
    main()

Using device: cuda
CUDA Version: 12.4
BitsAndBytes Version: 0.45.3
Initial GPU memory: 0 MB
Loading tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading teacher model in 8-bit...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Teacher model loaded. GPU memory: 3443 MB
Loading student model in 8-bit...
Student model loaded. GPU memory: 4875 MB


...

Memory cleared. GPU memory: 6091 MB


Epoch 2:  98%|█████████▊| 901/918 [04:15<00:06,  2.80it/s, loss=125]

Memory cleared. GPU memory: 6097 MB


Epoch 2: 100%|██████████| 918/918 [04:20<00:00,  3.53it/s, loss=1.77]


Epoch 2 average loss: 61.8027
GPU memory after epoch: 5916 MB
Saving 8-bit model...
Distilled model saved as 'distilled_llama_1b_8bit'.
Memory cleared. GPU memory: 2470 MB
Process completed successfully.


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
)
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

def generate_text(model, tokenizer, prompt, max_length=256, temperature=0.7, num_return_sequences=1):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

def evaluate_perplexity(model, tokenizer, dataset, batch_size=4):
    model.eval()

    # Tokenize the dataset
    def tokenize_function(examples):
        return tokenizer(examples["text"], truncation=True, max_length=512)
    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

    # Use a collator that pads batches (no masked language modeling needed)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    dataloader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)

    total_loss = 0.0
    total_tokens = 0
    for batch in tqdm(dataloader, desc="Evaluating Perplexity"):
        batch = {k: v.to(model.device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        # outputs.loss is the average loss per token; multiply by total tokens in batch
        loss = outputs.loss.item() * batch["input_ids"].numel()
        total_loss += loss
        total_tokens += batch["input_ids"].numel()
    avg_loss = total_loss / total_tokens
    ppl = np.exp(avg_loss)
    return ppl

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
    print("Using device:", device)

    teacher_model_id = "meta-llama/llama-3.2-3b"  # 8-bit teacher model
    student_model_id = "meta-llama/llama-3.2-1b"    # 8-bit student model
    distilled_model_path = "distilled_llama_1b_8bit"       # 8-bit distilled model saved locally

    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch.float16
    )

    tokenizer = AutoTokenizer.from_pretrained(teacher_model_id)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    teacher_model = AutoModelForCausalLM.from_pretrained(
        teacher_model_id,
        device_map="auto",
        quantization_config=quantization_config
    )
    teacher_model.eval()

    student_model = AutoModelForCausalLM.from_pretrained(
        student_model_id,
        device_map="auto",
        quantization_config=quantization_config
    )
    student_model.eval() 

    distilled_model = AutoModelForCausalLM.from_pretrained(
        distilled_model_path,
        device_map="auto",
        quantization_config=quantization_config
    )
    distilled_model.eval()

    print("\n--- Code Generation Evaluation ---")
    code_prompts = [
        "def add(a, b):\n    \"\"\"Return the sum of a and b.\"\"\"\n    ",
        "def factorial(n):\n    \"\"\"Return the factorial of n.\"\"\"\n    ",
        "def is_prime(n):\n    \"\"\"Return True if n is a prime number, False otherwise.\"\"\"\n    "
    ]

    for prompt in code_prompts:
        print("\nPrompt:")
        print(prompt)
        teacher_output = generate_text(teacher_model, tokenizer, prompt)[0]
        student_output = generate_text(student_model, tokenizer, prompt)[0]
        distilled_output = generate_text(distilled_model, tokenizer, prompt)[0]

        print("\nTeacher Model Output:")
        print(teacher_output)
        print("\nStudent Model Output:")
        print(student_output)
        print("\nDistilled Model Output:")
        print(distilled_output)

    print("\n--- Perplexity Evaluation on WikiText-2 ---")
    wikitext_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="validation")

    teacher_ppl = evaluate_perplexity(teacher_model, tokenizer, wikitext_dataset, batch_size=4)
    student_ppl = evaluate_perplexity(student_model, tokenizer, wikitext_dataset, batch_size=4)
    distilled_ppl = evaluate_perplexity(distilled_model, tokenizer, wikitext_dataset, batch_size=4)

    print("\nPerplexity Scores:")
    print(f"Teacher Model Perplexity: {teacher_ppl:.2f}")
    print(f"Student Model Perplexity: {student_ppl:.2f}")
    print(f"Distilled Model Perplexity: {distilled_ppl:.2f}")

if __name__ == "__main__":
    main()

Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:206: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- Code Generation Evaluation ---

Prompt:
def add(a, b):
    """Return the sum of a and b."""
    

Teacher Model Output:
def add(a, b):
    """Return the sum of a and b."""
    

Student Model Output:
def add(a, b):
    """Return the sum of a and b."""
    

Distilled Model Output:
def add(a, b):
    """Return the sum of a and b."""
    

Prompt:
def factorial(n):
    """Return the factorial of n."""
    

Teacher Model Output:
def factorial(n):
    """Return the factorial of n."""
    

Student Model Output:
def factorial(n):
    """Return the factorial of n."""
    

Distilled Model Output:
def factorial(n):
    """Return the factorial of n."""
    

Prompt:
def is_prime(n):
    """Return True if n is a prime number, False otherwise."""
    

Teacher Model Output:
def is_prime(n):
    """Return True if n is a prime number, False otherwise."""
    

Student Model Output:
def is_prime(n):
    """Return True if n is a prime number, False otherwise."""
    

Distilled Model Output:
d

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

Evaluating Perplexity: 100%|██████████| 940/940 [02:14<00:00,  6.96it/s]


Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

Evaluating Perplexity: 100%|██████████| 940/940 [01:18<00:00, 12.02it/s]


Perplexity Scores:
Teacher Model Perplexity: 16.57
Student Model Perplexity: 21.07
Distilled Model Perplexity: 20.32
